### Identify projects with activity in every months for analysis period

Filter projects (prs_selected is true) with (for each analysis month):
* 1 closed issue

In [29]:
repos = getProjects()

for repo in repos:    
    project = repo[0]
    init_period = repo[1]
    ci_service = repo[2]
    
    '''
    if ci_service is not None:
        
        previous_period = classifyProjectPR(project,analysisPoint,-1,-12)
        following_period = classifyProjectPR(project,analysisPoint,1,12)

        if previous_period is not None and following_period is not None:
            prs_period = countPRsPeriod(project,previous_period,following_period)
            setRQ2Classification(project,True,24,prs_period,previous_period,following_period)
            setRQ2Classification(project)
            print('PRs: {}  -  Project: {} - Init: {} - Finish: {}'.format(prs_period,project,previous_period,following_period))
        
        
    else:
    '''    
        
    end_period = getLastIssueDate(project)
    print('Project: {}  -  Init: {} - End: {}'.format(project,init_period,end_period))
    identifyIssuesActivity(project,init_period,end_period)

Project: antvis/L7  -  Init: 2019-08-13 06:49:11-03:00 - End: 2021-09-04 08:45:47-03:00
Project: atmos/heaven  -  Init: 2014-02-26 03:35:52-03:00 - End: 2019-01-08 05:39:13-03:00
Project: catarse/catarse  -  Init: 2012-12-04 17:34:31-03:00 - End: 2019-02-01 23:29:23-03:00
Project: github/developer.github.com  -  Init: 2015-11-02 06:35:04-03:00 - End: 2017-10-11 20:18:38-03:00
Project: gpac/gpac  -  Init: 2015-03-10 06:07:59-03:00 - End: 2021-09-02 12:56:23-03:00
Project: graphiti-api/graphiti  -  Init: 2019-01-06 16:34:11-03:00 - End: 2021-09-02 04:18:00-03:00
Project: hyperhq/hyperd  -  Init: 2015-08-14 06:17:40-03:00 - End: 2018-12-23 21:00:40-03:00
Project: idehub/react-native-google-analytics-bridge  -  Init: 2016-07-16 11:02:00-03:00 - End: 2019-03-08 13:34:35-03:00
Months: 12  -  Issues: 31  -  Project: idehub/react-native-google-analytics-bridge
Project: infernojs/inferno  -  Init: 2015-11-23 12:36:30-03:00 - End: 2021-08-15 23:57:25-03:00
Project: intelsdi-x/snap  -  Init: 2014

In [22]:
def identifyIssuesActivity(project,init,finish):
    search_date = analysis_init_date = init
    analysis_finish_date = None
    increment=1
    period = total_issues = 0
    
    while search_date <= finish:
        
        issues = selectIssues(project,search_date.month,search_date.year)
        if issues[0]>0:
            period += 1
            total_issues += issues[0]
            
        else:    
            break
             
                
        search_date = init + relativedelta(months=increment)
        increment +=1
        
        if search_date.month == finish.month and search_date.year == finish.year:
            search_date = finish
    
    if period > 11:
        print('Months: {}  -  Issues: {}  -  Project: {}'.format(period,total_issues,project))
        setIssuesClassification(project,True,period,total_issues)
    else:
        setIssuesClassification(project,False,period,total_issues)

### Aux Functions

In [2]:
import psycopg2
import os
import requests 
import time
import pytz    
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
def connectDB():
    f = open('/home/elieziosoares/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "postgres",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v3")

In [28]:
def getProjects():
    query = """SELECT repo_name, analysis_init,ci_service From PROJECTS WHERE prs_SELECTED is True and issues_selected is not true ORDER BY repo_name"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [1]:
def selectIssues(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """select COUNT(ID) from issue where  
        date_part('month', created_at) = %s
        AND date_part('year', created_at) = %s 
        AND repo_name like %s"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return row

In [25]:
def setIssuesClassification(repo_name,classification,period_max,qtd_issues_in_period):
    query = """UPDATE  projects 
        set issues_selected = %s,
        qtd_issues_months = %s,
        qtd_issues_in_period = %s
        WHERE repo_name like %s"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [classification,period_max,qtd_issues_in_period,repo_name])
    connection.commit()
    cursor.close()
    connection.close()

In [14]:
def getLastIssueDate(project):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT created_at from issue where  
        repo_name like %s AND created_at is not NULL
        order by created_at desc limit 1;"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [12]:
def selectLastIssueMonth(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT closed_at from issue where  
        date_part('month', created_at) = %s
        AND date_part('year', created_at) = %s 
        AND repo_name like %s
        order by closed_at desc limit 1;"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [13]:
def getFirstIssueDate(project):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT created_at from issue where  
        repo_name like %s
        order by created_at limit 1;"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [8]:
def selectFirstIssueMonth(project,date):
#def selectFirstPRMonth(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    #query = """SELECT closed_at from pullrequests where  
    #    date_part('month', closed_at) = %s
    #    AND date_part('year', closed_at) = %s 
    #    AND project_name like %s
    #    order by closed_at limit 1;"""
    
    #cursor.execute(query, [month,year,project])
    
    query = """SELECT closed_at from issue where  
        closed_at >= %s
        AND repo_name like %s
        order by closed_at limit 1;"""

    cursor.execute(query, [date,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [15]:
def selectIssues(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT COUNT(ID) from issue where  
        date_part('month', created_at) = %s
        AND date_part('year', created_at) = %s 
        AND repo_name like %s"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return row